Install npTDMS to actually pull in the TDMS files.



In [1]:
#import sys
#!{sys.executable} -m pip install nptdms

# At least on my system, requirements already satisfied.

Import the tdmsfile reader capabilities from nptdms

In [2]:

import time as time
import numpy as np
from nptdms import TdmsFile
from pprint import pprint
import os
from os import walk
import shutil
import pathlib
from pathlib import Path
import csv
from itertools import chain

startTime = time.time()


## Actually reading things in:

The file structure currently isn't really nested the way we want it to be. We need to know:

1. How many parts are in our TDMS group.
2. How many layers total are in our TDMS group.

Ideally, I'd like to be able to work with zipped folders, but there are no guarantees on that one.

First goal: Write a method that can find all of our folders witih TDMS files included.

Folder needs to have Slice00000#.tdms inside

So: 
Slice000001.tdms
Slice000002.tdms
Slice000003.tdms ...

...
Slice00000N.tdms
for N slices

The result will be a directory called "NAME, DATE, TDMS Files"
Inside: 
\*PART 1 NAME\*, \*PHD OR CAMERA\* TDMS Files
\*PART 2 NAME\*, \*PHD OR CAMERA\* TDMS Files
\*PART 3 NAME\*, \*PHD OR CAMERA\* TDMS Files
...
\*PART 1 NAME\*, \*PHD OR CAMERA\* TDMS Files

Inside each one of those will be:
Slice0001.csv
Slice0002.csv
...
Slice000N.csv

For the N slices.


In [3]:


#print("Example directory path: \nC:/Users/maxah/Documents/CDME/Sept 9 2019 TDMS Parts/TDMS\n")
testPath = "C:\\Users\\maxah\\Documents\\CDME\\Sept 9 2019 TDMS Parts\\TDMS"

print("Enter directory path. Forward or backward slash doesn't matter.\n")

time.sleep(.1) #just so that the output makes sense to the user in the correct order; otherwise,
#                     the input prompt would come first and be confusing.
yesNoChar = 'n'
dir_path = ""
numFiles = 0
while( 'y' not in yesNoChar and 'Y' not in yesNoChar):
    
    if(yesNoChar == 'n' or yesNoChar == "N"):
        dir_path = input("Example path: " + "C:\\Users\\maxah\\Documents\\CDME\\Sept 9 2019 TDMS Parts\\TDMS" + "\n")  
        if("testpath" in dir_path or "TESTPATH" in dir_path):
            dir_path = testPath
    
    #dir_path = testPath

    if('\\' in dir_path):
        newPath = dir_path.replace("\\","/")
        dir_path = newPath
    sourceTDMSDirectory = Path(dir_path)        

    print("The chosen directory is: " + dir_path)   
    
    fileNames = []
    for (dirpath, dirnames, filenames) in walk(sourceTDMSDirectory):
        fileNames.extend(filenames)
        break

    tdmsFiles = []
    for fileName in fileNames:
        if ".tdms_index" not in fileName:
            tdmsFiles.append(fileName)
            numFiles = numFiles + 1
    print("Here are the contents of that directory: \n")
    length = len(tdmsFiles)
    pprint(tdmsFiles)
    
    yesNoChar = input("If that is not the directory you want, hit 'n'. If that is the directory you want, hit 'y'.\n")
    
print("What are you going to call this project?\nEX: \"Nov 21 Build TDMS\"")
taskName = "Processed Stacks " + input("")
sourceTDMSDirectory = Path(dir_path)

"""
print("What do you want the batch size to be?\nRecommended values: 5 or 10. Higher batch size means higher memory usage.")
batchSize = int(input("Please enter a number: "))

"""
batchSize = 1

Enter directory path. Forward or backward slash doesn't matter.

Example path: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\TDMS
C:/Users/maxah/Documents/CDME/Sept 9 2019 TDMS Parts/TDMS
The chosen directory is: C:/Users/maxah/Documents/CDME/Sept 9 2019 TDMS Parts/TDMS
Here are the contents of that directory: 

['Slice00001.tdms',
 'Slice00002.tdms',
 'Slice00003.tdms',
 'Slice00004.tdms',
 'Slice00005.tdms']
If that is not the directory you want, hit 'n'. If that is the directory you want, hit 'y'.
y
What are you going to call this project?
EX: "Nov 21 Build TDMS"
Nov 26 Test 2


### Filepath now stored. 

Filepath is now stored and we have a handy dandy conversion from Windows file structure to a python readable format.
Let's create a list of everything in the directory.

Any ".tdms_index" files are created by using the National Instruments' "TDMS To Excel" tool, and can be ignored.
They will just be removed from the list.

In [4]:
# Make a map of the file names to new file locations; 

#Make a "Processed" directory as a subdirectory one folder up

directoryBuildTime = time.time()
#sourceTDMSDirectory is where we are collecting these files
firstObject = TdmsFile(str(sourceTDMSDirectory) + "\\" + str(tdmsFiles[0]))

parentDir = sourceTDMSDirectory.parent
newFolder = parentDir / str(taskName)
newFolder.mkdir(exist_ok=True, parents=True)

#Now that we have the folder: let's go in and make a subfolder for every group we've got in the TDMS files.
FolderDictionary = dict()
groups = firstObject.groups()
for part in groups:
    folderName = str(part)
    newDir = newFolder / folderName
    newDir.mkdir(exist_ok=True, parents=True)
    #print("\n \n" + str(part) + " \n" + str(newDir))
    
    FolderDictionary[str(part)] = str(newDir)
    
#pprint(FolderDictionary)
#Boom: we've now got a folder for every part, and a mapping of group name (part name) to folder name.

#Next step is to go into every TDMS's item, and write the
#associated data to a CSV inside of the folder for that name, with a good name for the slice.
#sourceTDMSDirectory

directoryBuildTime = time.time() - directoryBuildTime
print("Time to build directories was " + str(directoryBuildTime) + " seconds.\n")

Time to build directories was 2.4946281909942627 seconds.



In [5]:
#print(str(sourceTDMSDirectory) + "\\" + tdmsFiles[0])
#testTime = time.time()
openedSoFar = 0
numLayers = 0
tdmsObjects = []
numBatches = 0
populatedList = set()
sliceCounter = 0
#Outermost loop: controls the number of files written, total.
while numLayers < numFiles:
    
    #Gets files from TDMS files
    #   This one isn't the outermost because I want to manage memory better.
    for file in tdmsFiles:
        
        #opens a TDMS file. Very memory intensive.
        tdmsObjects.append(TdmsFile(str(sourceTDMSDirectory) + "\\" + str(file)))
        numLayers+=1
        
        #Once we've got a few of the TDMS files open, transform them into CSV's.
        #   Happens every time a batch is fill (IE: every 5th file)
        if(numLayers % batchSize == 0 or numLayers == numFiles):
            
            #all of the actual file writing will happen in here
            
            processTime = time.time()
            print("Block " + str(numBatches))
            print(str(tdmsObjects) + "\n")
            
            
            colNames = []
            populatedList = set()
            
            print(tdmsObjects)
            print(numLayers)
            
            for tdms in tdmsObjects:
                sliceCounter = sliceCounter + 1
                groups = tdms.groups()
                #print(tdms)
                for part in groups:
                    #get the data from each group's channel and make a CSV
                    channels = tdms.group_channels(part)
        
        
                    #make a 2D array, and populate it with the arrays in this loop.
                    groupCSV = []
                    areaCol = []
                    xCol = []
                    yCol = []
                    paramCol = []
                    intensityCol = []
                    laserCol = []
                    csvCount = 0
                    #copy each channel's data to its respective frame
        
                    for channel in channels:
            
                        names = []
                        for i in channels:
                            wordList = str(i).split("/")
                            name = wordList[-1]
                            name = name.strip(">")
                            name = name.strip("'")
                            names.append(name)
                        colNames = names    
                        #pprint(channel.data)
                        name = channel.channel
                        data = channel.data
            
                        if(names[0] in name):
                            areaCol = data
                        elif(names[1] in name):
                            xCol = data
                        elif(names[2] in name):
                            yCol = data
                        elif(names[3] in name):
                            paramCol = data
                        elif(names[4] in name):
                            intensityCol = data
                        elif(names[5] in name):
                            laserCol = data
                        groupCSV.append(data)
                        csvCount += 1
            
                    csvFileLocation = FolderDictionary[str(part)]
                    sliceFileName = str(csvFileLocation) + "\Slice000" + str(sliceCounter) + ".csv"
                    print("Writing file: " + str(sliceFileName) + "\n")
                    with open(sliceFileName, mode="w", newline = "") as csvfile:
                        wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL, dialect="excel")
                        wr.writerow(names)
                        if(len(areaCol > 1)):
                            for rows in range(0,len(areaCol) - 1):
                                #print(str(len(areaCol)) + "\n")
                                row = [areaCol[rows], xCol[rows], yCol[rows], paramCol[rows], intensityCol[rows], laserCol[rows]]
                                wr.writerow(row)
                            populatedList.add(part)
                
            
            processTime = time.time() - processTime
            print("Time for this batch was " + str(processTime) + " seconds.")
            print("Or " + str(processTime / sliceCounter) + " seconds per layer.\n")

            
            #end operation space
            #this declaration clears the memory for the tdms objects
            tdmsObjects = []
            numBatches = numBatches + 1
        
    

    
#endTime = time.time()

#timeLength = endTime - testTime
#this block takes roughly 13.49 seconds on my machine -- 
#print("All TDMS objects read in, took " + str(timeLength) + " seconds.\n" + str((timeLength)/5) + " seconds per layer.\n")
#or roughly 2.7 seconds per layer. That isn't ideal, 
#since it's the price we can stand to pay for every access of a TDMS object.

Block 0
[<nptdms.tdms.TdmsFile object at 0x000001A2A792CF88>, <nptdms.tdms.TdmsFile object at 0x000001A2A8929108>, <nptdms.tdms.TdmsFile object at 0x000001A2A78EEC08>]

[<nptdms.tdms.TdmsFile object at 0x000001A2A792CF88>, <nptdms.tdms.TdmsFile object at 0x000001A2A8929108>, <nptdms.tdms.TdmsFile object at 0x000001A2A78EEC08>]
3
Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00001_op1_calipar_box_contours-only_cl\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00001_op1_calipar_box_contours-only_cl\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00002_op2_caliper_box-contours-only_cl\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00002_op2_caliper_box-contours-only_cl\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 20

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00030_op1_ATSM_DIS_52902_f_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00031_op1_calipar_box_normal_2_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00031_op1_calipar_box_normal_2_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00032_op2_caliper_box-normal_2_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00032_op2_caliper_box-normal_2_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00033_op1_1mm_BOP_vert_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00033_op1_1m

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00060_Downskin Part v4arch_supports_cl\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00061_op1_2mm_BOP_vert_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00061_op1_2mm_BOP_vert_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00062_op2_2mm_BOP_vert_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00062_op2_2mm_BOP_vert_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00063_Downskin Part v3b_supports_1_cls\Slice0001.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00063_Downskin Part v3b_

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00019_op1_hatchoverlap_11_contour_1_cl\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00019_op1_hatchoverlap_11_contour_1_cl\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00020_op1_hatchoverlap_12_contour_1_cl\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00020_op1_hatchoverlap_12_contour_1_cl\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00021_op1_hatchoverlap_11_btm_1_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00021_op1_hatchoverlap_11_btm_1_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stack

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00049_op1_hatchoverlap_11_contour_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00049_op1_hatchoverlap_11_contour_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00050_op1_hatchoverlap_12_contour_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00050_op1_hatchoverlap_12_contour_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00051_op1_hatchoverlap_11_btm_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00051_op1_hatchoverlap_11_btm_cls\Slice0002.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26

...

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00013_op1_contouroverlap_supports_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00014_op1_contouroverlap_11_btm_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00014_op1_contouroverlap_11_btm_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00015_op1_contouroverlap_11_top_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00015_op1_contouroverlap_11_top_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00016_op1_contouroverlap_12_top_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 T

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00041_op1_Calibration block v2_4_outsi\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00042_op2_Calibration block v2_4_insid\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00042_op2_Calibration block v2_4_insid\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00043_op1_2mm_BOP_left_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00043_op1_2mm_BOP_left_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00044_op1_plus_hatches_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00044_op1_pl

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00071_op1_BL_Beam_Offset_V1_250-25-75_\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00072_op1_zstack_supports1_4_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00072_op1_zstack_supports1_4_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00073_op1_zstack1_4_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00073_op1_zstack1_4_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00074_op2_zstack1_4_cls\Slice0004.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00074_op2_zstack1_4_cls\Slice0004

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00025_op1_zstack_supports3_3_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00026_op1_zstack3_3_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00026_op1_zstack3_3_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00027_op2_zstack3_3_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00027_op2_zstack3_3_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00028_op1_cross_2_1_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00028_op1_cross_2_1_cls\Slice0005.csv

Writing file: C:\U

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00056_Downskin Part v4arch_1_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00056_Downskin Part v4arch_1_cls\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00057_op1_Downskin Part v3b_contours_c\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00057_op1_Downskin Part v3b_contours_c\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\0_00058_op2_Downskin Part v3b_hatch2_1_c\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 26 Test 2\1_00058_op2_Downskin Part v3b_hatch2_1_c\Slice0005.csv

Writing file: C:\Users\maxah\Documents\CDME\Sept 9 2019 TDMS Parts\Processed Stacks Nov 

Time information:
It's taking roughly 30 seconds per layer, but that's probably pretty variable.
If you want to do 800 layers then, we're looking at 2400 seconds or 40 minutes.
This might be the kind of script you set up and run and leave overnight once it's finished.
Speed may also depend on the computer it's being performed on -- the slowdown is likely the read/writes happening, so the 
hard disk in the desktops here at the CDME could be slower than my machine's SSD.

In [6]:
def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

def count_files(start_path = "."):
    total_files = 0
    return(len(os.listdir(start_path)))
    

#print(get_size(), 'bytes')





In [7]:
partsDeleted = 0
partsRemaining = 0
for k, v in FolderDictionary.items():
    byteSize = get_size(v)
    fileCount = count_files(v)
    minimumBytes = 62 * int(fileCount) #61 is the number of bytes contained in the headers.
    #If a folder has less than 62 bytes per file, then delete the folder.
    #Note: if there are some empty files in a folder, leave the entire thing for now.
    if (byteSize < minimumBytes):
        #delete the directory v, since many of the folders don't actually contain any data
        partsDeleted = partsDeleted + 1
        shutil.rmtree(v)
    else:
        partsRemaining = partsRemaining + 1

# Boom: now it only keeps those which have items inside.
print("Finished.\n")
pprint(populatedList)
        
endTime = time.time()
runTime = endTime - startTime
print("\n***FINISHED REPORT***")
print("\n" + "Runtime was: " + str(runTime) + " seconds.")
print("Number of layers written was: " + str(numLayers) + " layers.")
print("Number of parts with no data was: " + str(partsDeleted) + " parts.")
print("Number of parts with data was: " + str(partsRemaining) + " parts.")
print("Parts with data: ")

#input hold; causes the program to freeze before exiting.
exit = input("Finished: type the X key to exit.")
time.sleep(.01)
if("x" not in exit and "X" not in exit):
    input()

Finished.

{'0_00001_op1_calipar_box_contours-only_cl',
 '0_00003_op1_Calibration block v2_3_outsi',
 '0_00005_op1_zstack_supports2_2_cls',
 '0_00008_op1_contouroverlap_supports2_cls',
 '0_00013_op1_contouroverlap_supports_cls',
 '0_00025_op1_zstack_supports3_3_cls',
 '0_00028_op1_cross_2_1_cls',
 '0_00030_op1_ATSM_DIS_52902_f_cls',
 '0_00031_op1_calipar_box_normal_2_cls',
 '0_00033_op1_1mm_BOP_vert_cls',
 '0_00038_op1_num_supports_cls',
 '0_00039_op1_1mm_BOP_left_cls',
 '0_00041_op1_Calibration block v2_4_outsi',
 '0_00043_op1_2mm_BOP_left_cls',
 '0_00046_op1_plus_supports_cls',
 '0_00060_Downskin Part v4arch_supports_cl',
 '0_00061_op1_2mm_BOP_vert_cls',
 '0_00064_op1_zstack_supports_cls',
 '0_00071_op1_BL_Beam_Offset_V1_250-25-75_',
 '0_00072_op1_zstack_supports1_4_cls',
 '1_00002_op2_caliper_box-contours-only_cl',
 '1_00004_op2_Calibration block v2_3_insid',
 '1_00018_op2_hatchoverlap_supports_cls',
 '1_00029_op2_cross_2_1_cls',
 '1_00032_op2_caliper_box-normal_2_cls',
 '1_00034_op